In [1]:
import torch
import torch.nn.functional
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:

words = open('names.txt', 'r').read().splitlines()
words[:8]
len(words)

32033

In [3]:
chars = [chr(i) for i in range(ord('a'), ord('z')+1)]
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [8]:
X, Y = [], []
for w in words:
    chs = list(f"...{w}.")
    for i in range(len(chs)-3):
        x1, x2, x3, y =  chs[i:i+4]
        # print("".join([x1, x2, x3]), "--->", y)
        X.append((stoi[x1],stoi[x2],stoi[x3]))
        Y.append(stoi[y])

n = len(X)
g = torch.Generator().manual_seed(2147483647)
perm = torch.randperm(n, generator=g)

n_train = int(0.8 * n)
n_dev = int(0.1 * n)

Xtrain, Ytrain = torch.tensor([X[i] for i in perm[:n_train]], dtype=torch.long), torch.tensor([Y[i] for i in perm[:n_train]], dtype=torch.long)
Xdev, Ydev = torch.tensor([X[i] for i in perm[n_train:n_train + n_dev]], dtype=torch.long), torch.tensor([Y[i] for i in perm[n_train:n_train + n_dev]], dtype=torch.long)
Xtest, Ytest = torch.tensor([X[i] for i in perm[n_train + n_dev:]], dtype=torch.long), torch.tensor([Y[i] for i in perm[n_train + n_dev:]], dtype=torch.long)


In [9]:
print(Xtrain.shape, Ytrain.shape)
print(Xdev.shape, Ydev.shape)
print(Xtest.shape, Ytest.shape)

torch.Size([182516, 3]) torch.Size([182516])
torch.Size([22814, 3]) torch.Size([22814])
torch.Size([22816, 3]) torch.Size([22816])


In [16]:
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn((100), generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27)
parameters = [C,W1,b1,W2,b2]
nump = sum(p.nelement() for p in parameters)
for p in parameters:
    p.requires_grad=True

In [ ]:
import torch.nn.functional as F
for i in range(1):
    #minibatch
    ix = torch.randint(Xtrain.shape[0], (32,))  # (32, )
    
    # forward pass
    emb = C[Xtrain[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits)
    print(loss.item())

    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1
    for p in parameters:
        p += -lr * p.grad



tensor([[[-1.0724,  0.8418],
         [-1.0724,  0.8418],
         [ 1.5095, -2.6399]],

        [[-1.0724,  0.8418],
         [-1.0724,  0.8418],
         [-1.0724,  0.8418]],

        [[-1.0724,  0.8418],
         [-1.0724,  0.8418],
         [ 0.7992,  0.0315]]])
